In [1]:
import os.path
import sqlite3
import socket
import numpy as np
np.seterr(divide='ignore')


from torch.utils.data import Dataset

if socket.gethostname() == 'LTSSL-sKTPpP5Xl':
    data_dir = 'C:\\Users\\ams90\\PycharmProjects\\ConceptsBirds\\data'
elif socket.gethostname() == 'LAPTOP-NA88OLS1':
    data_dir = 'D:\\data\\caltecBirds\\CUB_200_2011'
else:
    data_dir = '/home/bwc/ams90/datasets/caltecBirds/CUB_200_2011'


In [2]:
conn = sqlite3.connect(database=os.path.join(data_dir, 'birds.db'))
conn.create_function("log", 1, np.log)
conn.create_function("exp", 1, np.exp)
cursor = conn.cursor()

In [3]:
classes      = {class_id: class_name     for (class_id, class_name)     in cursor.execute("select class_id, class_name from classes").fetchall()}
concepts     = {concept_id: concept_name for (concept_id, concept_name) in cursor.execute("select concept_id,concept_name from concepts").fetchall()}
attributes   = {attribute_id: attribute_id for (attribute_id, ) in cursor.execute("select attribute_id from attributes").fetchall()}
class_counts = {class_id: count for (class_id, count) in cursor.execute("select class_id, count(*) from images where trainset = 1 group by class_id").fetchall()}


In [65]:
count, correct, unknown, wrong = 0, 0, 0, 0
loop = conn.cursor()
dataset = 1

count, correct = 0, 0
for image_id, class_id in loop.execute("select image_id, class_id from images where trainset = ?", (dataset,)).fetchall():
    
   #present_attribute_ids = [row[0] for row in cursor.execute("select attribute_id from image_attributes where present = 1 and image_id = ?", (image_id,)).fetchall()]
    
    class_log_probabilities = [row[0] for row in cursor.execute("""select sum(ia.present * nbc.log_likelihood  + (1-ia.present) * nbc.log_likelihood_conjugate)
                                                                 from     nb_attribute_counts nbc,
                                                                          image_attributes ia
                                                                 where    nbc.attribute_id = ia.attribute_id
                                                                 and      nbc.trainset     = ?
                                                                 and      ia.image_id      = ?
                                                                 group by nbc.class_id
                                                                 order by nbc.class_id """, (dataset, image_id,)).fetchall()]
    
    class_probabilities = np.exp(class_log_probabilities)
    class_probabilities = [field for field in class_probabilities]
   

    
    
    total_prob = sum(class_probabilities)       
    if total_prob > 0:
        norm_probabilities  = [probability / total_prob for probability in class_probabilities]
        max_prob = max(norm_probabilities)
        pred_class_id = norm_probabilities.index(max_prob) + 1
        if pred_class_id == class_id: correct += 1
       
    
    count += 1

    print(image_id, class_id, pred_class_id, correct, round(correct/count,3))
    

loop.close()




2 1 1 1 1.0
4 1 1 2 1.0
5 1 1 3 1.0
7 1 1 4 1.0
8 1 1 5 1.0
9 1 1 6 1.0
11 1 1 7 1.0
15 1 1 8 1.0
18 1 1 9 1.0
20 1 1 10 1.0
21 1 1 11 1.0
23 1 1 12 1.0
30 1 1 13 1.0
31 1 1 14 1.0
32 1 1 15 1.0
35 1 1 16 1.0
36 1 1 17 1.0
37 1 1 18 1.0
40 1 1 19 1.0
41 1 1 20 1.0
42 1 1 21 1.0
43 1 1 22 1.0
46 1 1 23 1.0
49 1 1 24 1.0
51 1 1 25 1.0
53 1 1 26 1.0
55 1 1 27 1.0
56 1 1 28 1.0
58 1 1 29 1.0
60 1 1 30 1.0
62 2 1 30 0.968
64 2 1 30 0.938
66 2 1 30 0.909
67 2 1 30 0.882
69 2 1 30 0.857
72 2 1 30 0.833
73 2 1 30 0.811
74 2 1 30 0.789
75 2 1 30 0.769
77 2 1 30 0.75
78 2 1 30 0.732
80 2 1 30 0.714
81 2 1 30 0.698
84 2 1 30 0.682
86 2 1 30 0.667
89 2 1 30 0.652
96 2 1 30 0.638
97 2 1 30 0.625
99 2 1 30 0.612
102 2 1 30 0.6
104 2 28 30 0.588
105 2 1 30 0.577
109 2 1 30 0.566
110 2 1 30 0.556
111 2 1 30 0.545
113 2 1 30 0.536
115 2 1 30 0.526
116 2 1 30 0.517
117 2 1 30 0.508
120 2 1 30 0.5
121 3 1 30 0.492
122 3 1 30 0.484
123 3 1 30 0.476
124 3 1 30 0.469
125 3 1 30 0.462
127 3 1 30 0.455
128 3 

KeyboardInterrupt: 

In [67]:
norm_probabilities

[0.005115838337379157,
 0.004950811294237894,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.004950811294237894,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.003135513819684,
 0.005115838337379157,
 0.004950811294237894,
 0.004785784251096631,
 0.004950811294237894,
 0.004455730164814105,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.004950811294237894,
 0.004785784251096631,
 0.005115838337379157,
 0.005115838337379157,
 0.005115838337379157,
 0.00511583833

In [53]:
 alpha = 1
 z=   cursor.execute("""
                      select   i.class_id, 
                               i.trainset, 
                               ia.attribute_id, 
                               count(ia.attribute_id),
                               log((? +                   sum(ia.present)) / ((2.0 * ?) +  ncc.class_count)) ,
                               log((? + ncc.class_count - sum(ia.present)) / ((2.0 * ?) +  ncc.class_count))
                      from     images i,
                               image_attributes ia,
                               nb_class_counts  ncc
                      where    i.image_id = ia.image_id
                      and      ncc.class_id = i.class_id
                      and      ncc.trainset = i.trainset
                    
                      group by i.class_id, 
                               i.trainset, 
                               ia.attribute_id""", (alpha, alpha, alpha, alpha)).fetchall()

In [55]:
len(z)/200

624.0

In [28]:
cursor.execute("""select count(*)
                                                                 from     
                                                                          image_attributes ia
                                                                 
                                                                 where      ia.image_id      = ?
                                                                 group by nbc.class_id
                                                                 order by nbc.class_id """, (2,)).fetchall()

OperationalError: no such column: nbc.class_id